In [1]:
import numpy as np
import pandas as pd
pd.set_option('max_columns', 100)
pd.set_option('max_rows', 100)
from tqdm import tqdm
tqdm.pandas(desc='pandas bar')
import pickle
from sklearn.preprocessing import LabelEncoder

# 读数据
item读取有些麻烦, 需要按行来读. 其他数据正常使用pandas来读.

In [2]:
data_path = r'/workspace/mdls_test/contests/yidian/'
user = pd.read_table(data_path+'user_info.txt', sep='\t', names=['user_id', 'user_device', 'user_system', 'user_province', 'user_city', 'user_age', 'user_gender'])
#item = pd.read_table(data_path+'doc_info.txt', sep='\t', names=['item_id', 'item_title', 'item_time', 'item_picture', 'item_cluster1', 'item_cluster2', 'item_keywords'])
train = pd.read_table(data_path+'train_data.txt', sep='\t', names=['user_id', 'item_id', 'time', 'network', 'refresh', 'position', 'label', 'duration'])
test = pd.read_table(data_path+'test_data.txt', sep='\t', names=['id', 'user_id', 'item_id', 'time', 'network', 'refresh'])

In [3]:
with open(data_path+'doc_info.txt', 'r') as f:
    lines = f.readlines()

item = []
for line in tqdm(lines):
    unit = line.split('\t')
    if len(unit)<7:
        for _ in range(7-len(unit)):
            unit.append(np.nan)
    item.append(unit)
#item = np.array(item)

100%|██████████| 633391/633391 [00:03<00:00, 202792.70it/s]


In [4]:
item = pd.DataFrame(item, columns=['item_id', 'item_title', 'item_time', 'item_picture', 'item_cluster1', 'item_cluster2', 'item_keywords'])
item['item_id'] = item['item_id'].astype('int64')

## 加入统计特征

为了避免特征穿越，给线下validation重做user统计特征，方法是，用的train是文末处理后的新train

In [47]:
# 这个框一开始先不跑，不跑这个加统计特征是给线上用的，等生成json后，再返回这里用来生成线下统计特征, 再跑一边处理生成一个user_feature文件
# train = pd.read_table(data_path+'train_data.txt', sep='\t', names=['user_id', 'item_id', 'time', 'network', 'refresh', 'position', 'label', 'duration'])
validation = train.loc[train['time']>1625500811000]
train = train.loc[set(train.index) - set(validation.index)]

In [59]:
train

,user_id,item_id,time,network,refresh,position,label,duration
0,1000014754,463510256,1624843756147,5,0,16,0,0
1,1000014754,463852707,1624843756147,5,0,13,1,80
2,1000014754,464757134,1625052999841,5,0,13,1,1050
3,1000014754,464617167,1625052999841,5,0,16,1,286
4,1000014754,465426190,1625382421168,5,0,5,0,0
...,...,...,...,...,...,...,...,...
189766954,999938860,465384048,1625269599623,2,2,18,0,0
189766955,999938860,464821689,1625269568356,2,1,15,1,96
189766956,999938860,464835275,1625269568356,2,1,13,0,0
189766957,999938860,463270375,1624747290961,2,7,67,0,0


In [60]:
groups = train.groupby(by='user_id')
inter_count = groups['item_id'].count() # 用户交互过的文章数

In [61]:
refresh = groups['refresh'].agg(mean='mean', max='max', min='min') # 用户刷新次数的均值、最大值和最小值
refresh

,mean,max,min
user_id,,,
17340,5.595041,20,0
394666,1.454545,3,0
413322,2.000000,2,2
450280,2.937500,9,0
456646,1.653226,6,0
...,...,...,...
2447156416,0.000000,0,0
2447156430,0.000000,0,0
2447156434,0.800000,2,0


In [62]:
consumption = groups['duration'].agg(mean='mean', max='max', min='min') # 用户消费时长的均值、最大值和最小值
consumption

,mean,max,min
user_id,,,
17340,31.080579,399,0
394666,63.000000,268,0
413322,0.000000,0,0
450280,8.656250,94,0
456646,5.846774,88,0
...,...,...,...
2447156416,0.000000,0,0
2447156430,0.000000,0,0
2447156434,0.000000,0,0


In [63]:
# 统计特征新生成一个dataframe
userIds = train['user_id'].unique()
users = pd.DataFrame(userIds)
users.columns = ['user_id']
users = pd.merge(users, inter_count, on='user_id')
users.columns = ['user_id', 'inter_count']
users = pd.merge(users, refresh, on='user_id')
users.columns = ['user_id', 'inter_countt', 'refresh_mean', 'refresh_max', 'refresh_min']
users = pd.merge(users, consumption, on='user_id')
users.columns = ['user_id', 'inter_count', 'refresh_mean', 'refresh_max', 'refresh_min','consumption_mean', 'consumption_max', 'consumption_min']
users

,user_id,inter_count,refresh_mean,refresh_max,refresh_min,consumption_mean,consumption_max,consumption_min
0,1000014754,32,0.062500,1,0,128.750000,1050,0
1,1000026896,1282,8.169267,28,0,9.636505,422,-1
2,1000303258,25,0.960000,3,0,0.000000,0,0
3,1000346468,690,7.201449,26,0,35.450725,376,0
4,1000353866,1,1.000000,1,1,0.000000,0,0
...,...,...,...,...,...,...,...,...
1410162,999797434,2621,8.946967,43,0,8.565815,486,0
1410163,999838410,53,1.849057,6,0,91.396226,838,0
1410164,999873376,193,2.507772,11,0,0.186528,22,0
1410165,999929842,10,0.700000,1,0,40.700000,297,0


In [64]:
# 归一化
users[['inter_count', 'refresh_mean', 'refresh_max', 'refresh_min','consumption_mean', 'consumption_max', 'consumption_min']] = users[['inter_count', 'refresh_mean', 'refresh_max', 'refresh_min','consumption_mean', 'consumption_max', 'consumption_min']].apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))


In [58]:
users

,user_id,inter_count,refresh_mean,refresh_max,refresh_min,consumption_mean,consumption_max,consumption_min
0,1000014754,0.001140,0.000214,0.001538,0.000000,9.657535e-05,1.697036e-06,0.000422
1,1000026896,0.052057,0.029800,0.053846,0.000000,8.605101e-06,6.830124e-07,0.000000
2,1000303258,0.000882,0.003288,0.004615,0.000000,7.443187e-07,1.614687e-09,0.000422
3,1000346468,0.027168,0.023686,0.040000,0.000000,2.736679e-05,6.087369e-07,0.000422
4,1000353866,0.000000,0.003425,0.001538,0.003676,7.443187e-07,1.614687e-09,0.000422
...,...,...,...,...,...,...,...,...
1478689,999797434,0.103746,0.030558,0.066154,0.000000,7.775665e-06,7.863524e-07,0.000000
1478690,999838410,0.001912,0.006332,0.009231,0.000000,6.877224e-05,1.354722e-06,0.000422
1478691,999873376,0.007242,0.008406,0.016923,0.000000,8.796493e-07,3.713779e-08,0.000422
1478692,999929842,0.000331,0.002397,0.001538,0.000000,3.103809e-05,4.811766e-07,0.000422


In [65]:
users.fillna(0)
users

,user_id,inter_count,refresh_mean,refresh_max,refresh_min,consumption_mean,consumption_max,consumption_min
0,1000014754,0.001205,0.000214,0.001538,0.000000,8.945289e-05,1.697036e-06,0.000422
1,1000026896,0.049784,0.027977,0.043077,0.000000,7.333073e-06,6.830124e-07,0.000000
2,1000303258,0.000933,0.003288,0.004615,0.000000,6.894250e-07,1.614687e-09,0.000422
3,1000346468,0.026777,0.024663,0.040000,0.000000,2.513004e-05,6.087369e-07,0.000422
4,1000353866,0.000000,0.003425,0.001538,0.003676,6.894250e-07,1.614687e-09,0.000422
...,...,...,...,...,...,...,...,...
1410162,999797434,0.101823,0.030641,0.066154,0.000000,6.594911e-06,7.863524e-07,0.000422
1410163,999838410,0.002021,0.006332,0.009231,0.000000,6.370027e-05,1.354722e-06,0.000422
1410164,999873376,0.007462,0.008588,0.016923,0.000000,8.180224e-07,3.713779e-08,0.000422
1410165,999929842,0.000350,0.002397,0.001538,0.000000,2.874902e-05,4.811766e-07,0.000422


In [66]:
#分桶
def fen(x):
    if x==0:
        return 0
    elif x<=0.1:
        return 1
    elif x<=0.2:
        return 2
    elif x<=0.3:
        return 3
    elif x<=0.4:
        return 4
    elif x<=0.5:
        return 5
    elif x<=0.6:
        return 6
    elif x<=0.7:
        return 7
    elif x<=0.8:
        return 8
    elif x<=0.9:
        return 9
    else:
        return 10

users['inter_count'] = users['inter_count'].apply(lambda x: fen(x))
users['refresh_mean'] = users['refresh_mean'].apply(lambda x: fen(x))
users['refresh_max'] = users['refresh_max'].apply(lambda x: fen(x))
users['refresh_min'] = users['refresh_min'].apply(lambda x: fen(x))
users['consumption_mean'] = users['consumption_mean'].apply(lambda x: fen(x))
users['consumption_max'] = users['consumption_max'].apply(lambda x: fen(x))
users['consumption_min'] = users['consumption_min'].apply(lambda x: fen(x))
users

,user_id,inter_count,refresh_mean,refresh_max,refresh_min,consumption_mean,consumption_max,consumption_min
0,1000014754,1,1,1,0,1,1,1
1,1000026896,1,1,1,0,1,1,0
2,1000303258,1,1,1,0,1,1,1
3,1000346468,1,1,1,0,1,1,1
4,1000353866,0,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...
1410162,999797434,2,1,1,0,1,1,1
1410163,999838410,1,1,1,0,1,1,1
1410164,999873376,1,1,1,0,1,1,1
1410165,999929842,1,1,1,0,1,1,1


In [67]:
users['inter_count'].value_counts()

1     1150982
0      256164
2        2818
3         163
4          30
5           5
6           2
8           2
10          1
Name: inter_count, dtype: int64

In [15]:
import pickle as pkl
with open("user_statis.pkl", 'wb') as f:
    pkl.dump(users, f)
    f.close()

# 检查数据

# 处理数据

## 处理user特征

In [56]:
user

,user_id,user_device,user_system,user_province,user_city,user_age,user_gender
0,1000372820,2472,0,263,501,"[0.404616, 0.059027, 0.51655, 0.019806]","[0.051339, 0.948661]"
1,1000652892,1715,0,280,442,"[0.615458, 0.086233, 0.141408, 0.156901]","[0.280295, 0.719705]"
2,1000908852,1394,0,219,356,"[0.123255, 0.208225, 0.298089, 0.370431]","[0.0, 1.0]"
3,1001168798,2879,1,215,347,"[0.436296, 0.48937, 0.06156, 0.012773]","[0.87071, 0.12929]"
4,1001305614,1297,0,276,662,"[0.006632, 0.043408, 0.350842, 0.599118]","[0.0, 1.0]"
...,...,...,...,...,...,...,...
1538379,999798184,456,0,244,522,"[0.375362, 0.189298, 0.238601, 0.196739]","[0.950213, 0.049787]"
1538380,999813672,443,0,263,501,"[0.225201, 0.16979, 0.171753, 0.433256]","[0.298108, 0.701892]"
1538381,999859618,1023,0,281,416,"[0.838753, 0.04195, 0.029119, 0.090179]","[0.950213, 0.049787]"
1538382,999873694,1657,0,259,463,"[0.093117, 0.280029, 0.396015, 0.230839]","[0.103819, 0.896181]"


In [ ]:
# 填补空值
user['user_age'].fillna('A_0_24:0.25,A_25_29:0.25,A_30_39:0.25,A_40+:0.25', inplace=True)
user['user_gender'].fillna('female:0.5,male:0.5', inplace=True)

In [13]:
def get_multi_hot(x, k):
    str_list = x.split(',')
    res = [float(s.split(':')[1]) for s in str_list]
    for _ in range(k-len(res)):
        print('加0')
        res.append(0)
    if len(res) == 16:
        res = np.array(res)
        res = res[[0,4,8,12]]
        res = list(res)
    if (k == 2) & (len(res) == 4):
        res = np.array(res)
        res = res[[0,2]]
        res = list(res)
    if len(res) != k:
        print(x)
    return res

In [14]:
# 将age和gender都处理成列表的形式
user['new_age'] = user['user_age'].progress_apply(lambda x: get_multi_hot(x, 4))
user['new_gender'] = user['user_gender'].progress_apply(lambda x: get_multi_hot(x, 2))

pandas bar:  15%|█▌        | 233559/1538384 [00:00<00:03, 343255.36it/s]

加0


pandas bar:  24%|██▍       | 373243/1538384 [00:01<00:03, 347510.61it/s]

加0
加0


pandas bar:  45%|████▍     | 686357/1538384 [00:02<00:02, 325776.09it/s]

...

加0
加0
加0
加0


pandas bar:  94%|█████████▍| 1451168/1538384 [00:05<00:00, 331241.89it/s]

加0
加0


pandas bar: 100%|██████████| 1538384/1538384 [00:05<00:00, 281943.81it/s]


加0


In [15]:
user.drop(['user_age', 'user_gender'], axis=1, inplace=True)
user.rename(columns={'new_age': 'user_age', 'new_gender': 'user_gender'}, inplace=True)

In [16]:
user[['user_device', 'user_system', 'user_province', 'user_city']] = user[['user_device', 'user_system', 'user_province', 'user_city']].fillna('nan')

In [11]:
user.isnull().sum()

user_id          0
user_device      0
user_system      0
user_province    0
user_city        0
user_age         0
user_gender      0
dtype: int64

In [17]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
user[['user_device', 'user_system', 'user_province', 'user_city']] = user[['user_device', 'user_system', 'user_province', 'user_city']].apply(encoder.fit_transform)

In [70]:
user[['user_id','user_device', 'user_system', 'user_province', 'user_city']].nunique()

user_id          1538384
user_device         3096
user_system            3
user_province        328
user_city            768
dtype: int64

In [71]:
# 合并user和users
new_user = pd.merge(user,users,on='user_id',how='left')
new_user

,user_id,user_device,user_system,user_province,user_city,user_age,user_gender,inter_count,refresh_mean,refresh_max,refresh_min,consumption_mean,consumption_max,consumption_min
0,1000372820,2472,0,263,501,"[0.404616, 0.059027, 0.51655, 0.019806]","[0.051339, 0.948661]",2.0,1.0,1.0,0.0,1.0,1.0,0.0
1,1000652892,1715,0,280,442,"[0.615458, 0.086233, 0.141408, 0.156901]","[0.280295, 0.719705]",1.0,1.0,1.0,0.0,1.0,1.0,1.0
2,1000908852,1394,0,219,356,"[0.123255, 0.208225, 0.298089, 0.370431]","[0.0, 1.0]",1.0,1.0,1.0,0.0,1.0,1.0,0.0
3,1001168798,2879,1,215,347,"[0.436296, 0.48937, 0.06156, 0.012773]","[0.87071, 0.12929]",1.0,1.0,1.0,0.0,1.0,1.0,1.0
4,1001305614,1297,0,276,662,"[0.006632, 0.043408, 0.350842, 0.599118]","[0.0, 1.0]",1.0,1.0,1.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1538379,999798184,456,0,244,522,"[0.375362, 0.189298, 0.238601, 0.196739]","[0.950213, 0.049787]",1.0,1.0,1.0,0.0,1.0,1.0,1.0
1538380,999813672,443,0,263,501,"[0.225201, 0.16979, 0.171753, 0.433256]","[0.298108, 0.701892]",1.0,1.0,1.0,0.0,1.0,1.0,0.0
1538381,999859618,1023,0,281,416,"[0.838753, 0.04195, 0.029119, 0.090179]","[0.950213, 0.049787]",1.0,1.0,2.0,0.0,1.0,1.0,1.0
1538382,999873694,1657,0,259,463,"[0.093117, 0.280029, 0.396015, 0.230839]","[0.103819, 0.896181]",1.0,1.0,1.0,0.0,1.0,1.0,1.0


In [72]:
# 填补空缺值
new_user[['inter_count', 'refresh_mean', 'refresh_max', 'refresh_min','consumption_mean', 'consumption_max', 'consumption_min']] = new_user[['inter_count', 'refresh_mean', 'refresh_max', 'refresh_min','consumption_mean', 'consumption_max', 'consumption_min']].fillna(0)

In [73]:
new_user.isnull().sum()

user_id             0
user_device         0
user_system         0
user_province       0
user_city           0
user_age            0
user_gender         0
inter_count         0
refresh_mean        0
refresh_max         0
refresh_min         0
consumption_mean    0
consumption_max     0
consumption_min     0
dtype: int64

In [22]:
# df['列名'].value_counts()['某个值']
new_user['consumption_min'].value_counts()

1.0     1412228
0.0      120817
2.0        4369
3.0         856
4.0         108
5.0           4
8.0           1
10.0          1
Name: consumption_min, dtype: int64

In [21]:
new_user['consumption_max'].value_counts()

1.0     1478249
0.0       60134
10.0          1
Name: consumption_max, dtype: int64

In [24]:
new_user['consumption_mean'].value_counts()

1.0     1478249
0.0       60134
10.0          1
Name: consumption_mean, dtype: int64

In [25]:
uid_dict = dict(zip(new_user.user_id, new_user.index))
new_user['new_id'] = new_user.index
new_user.drop(['user_id'], axis=1, inplace=True)
new_user.rename(columns={'new_id': 'user_id'}, inplace=True)
new_user = new_user[['user_id', 'user_device', 'user_system', 'user_province', 'user_city','inter_count', 'refresh_mean', 'refresh_max', 'refresh_min','consumption_mean', 'consumption_max', 'consumption_min', 'user_gender', 'user_age']]
new_user[['user_id', 'user_device', 'user_system', 'user_province', 'user_city','inter_count', 'refresh_mean', 'refresh_max', 'refresh_min','consumption_mean', 'consumption_max', 'consumption_min']].nunique()

user_id             1538384
user_device            3096
user_system               3
user_province           328
user_city               768
inter_count              10
refresh_mean             10
refresh_max              11
refresh_min               8
consumption_mean          3
consumption_max           3
consumption_min           8
dtype: int64

## 根据需要保存user_feature
user_feature2-2 给线上,  user_feature2-3 给线下

In [26]:
with open(r'./user_feature2-2.pkl', 'wb') as f:
    pickle.dump(new_user, f, pickle.HIGHEST_PROTOCOL)

In [74]:
with open(r'./user_feature2-3.pkl', 'wb') as f:
    pickle.dump(new_user, f, pickle.HIGHEST_PROTOCOL)

## 处理item特征

In [36]:
item.head(10)

,item_id,item_title,item_time,item_picture,item_cluster1,item_cluster2,item_keywords
0,349635709,"拿到c1驾照后,实习期扣分了会怎样?扣12分驾照会吊销么?",1572519971000,9,汽车,汽车/用车,"上班族:8.469502,买车:8.137443,二手车:9.022247,副页:11.21..."
1,361653323,"疫情谣言粉碎机丨接种新冠疫苗后用麻药或致死?盘点最新疫情谣言,别被忽悠了",1624522285000,1,健康,健康/疾病防护治疗及西医用药,"医生:14.760494,吸烟:16.474872,板蓝根:15.597788,板蓝根^^熏..."
2,426732705,"实拍本田飞度:空间真大,8万出头工薪族可选,但内饰能忍?",1610808303000,9,汽车,汽车/买车,"155n:8.979802,polo:7.951116,中控台:5.954278,中网:7...."
3,430221183,搭载135kw电机比亚迪秦plus纯电动版外观更精致,1612581556000,2,汽车,汽车/买车,"etc:12.055207,代表:8.878175,内饰:5.342025,刀片:9.453..."
4,441756326,【提车作业】不顾他人眼光帕萨特phev俘获30老男人浪子心,1618825835000,23,汽车,汽车/买车,"丰田凯美瑞:12.772149,充电器:8.394001,品牌:8.436843,城市:7...."
5,443485341,"魏延有反骨之心都能重用,赵云忠心为什么却不被重用?",1619484501000,4,历史,历史/中国史,"三国:8.979797,五虎将:13.072728,人才:7.532783,保镖:6.811..."
6,447124796,"高考志愿|14个省份新高考录取有变化,这些专业傻傻分不清楚,填志愿看仔细",1624506618000,1,教育,教育/高考,"兴趣:11.558689,家长:11.350382,就业:12.176434,考生:11.5..."
7,448023100,三国煮酒论英雄,1624601704000,1,文化艺术,文化艺术/读书,"历史:9.184261,故事:9.405677\n"
8,452701283,懂我所需—吉利帝豪,1622081779000,19,汽车,汽车/买车,"ec7:14.355725,交互:10.203005,人机交互:10.540608,人机交互..."
9,452933213,"美丽的非洲长河?可以从这条河中,人的形象与命运?",1622121296000,3,旅游,旅游/旅游攻略,"东非:8.944922,东非高原:11.758570,两河:9.025832,伊兹:7.49..."


In [37]:
item['item_cluster2'].nunique()

369

In [27]:
item = item[['item_id', 'item_picture', 'item_cluster1', 'item_cluster2', 'item_keywords']]
item[['item_picture', 'item_cluster1', 'item_cluster2', 'item_keywords']] = item[['item_picture', 'item_cluster1', 'item_cluster2', 'item_keywords']].fillna('nan')

In [28]:
encoder = LabelEncoder()

item[['item_picture', 'item_cluster1', 'item_cluster2']] = item[['item_picture', 'item_cluster1', 'item_cluster2']].apply(encoder.fit_transform)

In [29]:
# 处理关键字信息
def chuli_k(x):
    if x == 'nan':
        return x
    else:
        y = x.split(',')
        z1 = [] # 存概率
        z2 = [] # 存keyword
        for i in range(len(y)): 
            if len(y[i].split(':'))==2:
                # 还要去掉^^
                word = y[i].split(':')[0].replace('^^','')
                if word != '':
                    z2.append(word)
                    z1.append(y[i].split(':')[1])
            else:
                if ':' in y[i]:
                    # 异形^^:^^火力^^小队:15.721525 处理这种情况
                    word = y[i].split(':',1)[1].split(':')[0].replace('^^','')
                    if word != '':
                        z2.append(word)
                        z1.append(y[i].split(':',1)[1].split(':')[1])
        return z2
item['keywords'] = item['item_keywords'].apply(lambda x: chuli_k(x))

In [30]:
# 处理关键字信息,得到每个的概率
def chuli_kp(x):
    if x == 'nan':
        return x
    else:
        y = x.split(',')
        z1 = [] # 存概率
        z2 = [] # 存keyword
        for i in range(len(y)): 
            if len(y[i].split(':'))==2:
                word = y[i].split(':')[0].replace('^^','')
                if word != '':
                    #z2.append(word)
                    z1.append(y[i].split(':')[1])
            else:
                if ':' in y[i]:
                    # 异形^^:^^火力^^小队:15.721525 处理这种情况
                    word = y[i].split(':',1)[1].split(':')[0].replace('^^','')
                    if word != '':
                        #z2.append(word)
                        z1.append(y[i].split(':',1)[1].split(':')[1])
        return z1
item['keywords_p'] = item['item_keywords'].apply(lambda x: chuli_kp(x))


In [31]:
# 先得到keywords
keywords = []
def get_k(x):
    if x != 'nan':
        keywords.extend(x)
item['keywords'].apply(lambda x: get_k(x))
keywords = set(keywords)
len(keywords)

1042572

In [32]:
# 对keywords进行label_encode
# 处理关键字信息，并得到encode的字典,label_encode时间太久了，用dict来做
keywords = list(keywords)
ind = list(range(1,len(keywords)+1))
keys = dict(zip(keywords,ind))
def chuli_kk(x):
    if x == 'nan':
        return x
    else:
        z = []
        for i in range(len(x)):
            z.append(keys[x[i]])
        return z
item['keywords'] = item['keywords'].apply(lambda x: chuli_kk(x))
item

,item_id,item_picture,item_cluster1,item_cluster2,item_keywords,keywords,keywords_p
0,349635709,145,26,226,"上班族:8.469502,买车:8.137443,二手车:9.022247,副页:11.21...","[634959, 656334, 1018555, 439496, 775027, 3751...","[8.469502, 8.137443, 9.022247, 11.218712, 5.10..."
1,361653323,2,5,57,"医生:14.760494,吸烟:16.474872,板蓝根:15.597788,板蓝根^^熏...","[927634, 449604, 284835, 911346, 875023, 53547...","[14.760494, 16.474872, 15.597788, 11.979615, 1..."
2,426732705,145,26,221,"155n:8.979802,polo:7.951116,中控台:5.954278,中网:7....","[818814, 756458, 800423, 721026, 877816, 10327...","[8.979802, 7.951116, 5.954278, 7.046073, 6.572..."
3,430221183,68,26,221,"etc:12.055207,代表:8.878175,内饰:5.342025,刀片:9.453...","[106512, 421744, 434700, 321587, 1018109, 9948...","[12.055207, 8.878175, 5.342025, 9.453552, 12.5..."
4,441756326,72,26,221,"丰田凯美瑞:12.772149,充电器:8.394001,品牌:8.436843,城市:7....","[125293, 368396, 385348, 737465, 832885, 97789...","[12.772149, 8.394001, 8.436843, 7.750292, 9.84..."
...,...,...,...,...,...,...,...
633386,467277215,134,31,264,"人因:6.528161,健康:6.471880,儿童:7.514914,儿童^^白血病:10...","[304638, 720573, 742535, 562460, 834129, 44684...","[6.528161, 6.471880, 7.514914, 10.299104, 9.16..."
633387,467277413,68,4,33,"体力:5.064578,吕钦:14.577604,唐丹:24.509433,唐丹^^男女:8...","[89172, 241425, 20402, 406868, 747256, 694856,...","[5.064578, 14.577604, 24.509433, 8.474139, 10...."
633388,467277503,123,19,180,"伊朗:7.489791,刷釉:18.557439,北宋:8.061633,博物馆:8.413...","[810303, 641660, 43629, 798206, 919699, 756955...","[7.489791, 18.557439, 8.061633, 8.413115, 11.6..."
633389,467278115,3,4,7,"cj:16.306715,三巨头:9.342195,争冠:8.706187,全明星:10.4...","[731390, 587774, 38673, 738788, 1039491, 59506...","[16.306715, 9.342195, 8.706187, 10.458268, 8.1..."


In [29]:
# 查看最长的keywordlist长度
item['keywords'].apply(lambda x: len(x)).max()

40

In [33]:
item = item[['item_id', 'item_picture', 'item_cluster1', 'item_cluster2', 'keywords', 'keywords_p']]

In [17]:
item['item_cluster1'].nunique()

44

In [21]:
item.isnull().sum()

item_id          0
item_picture     0
item_cluster1    0
item_cluster2    0
keywords         0
keywords_p       0
dtype: int64

In [34]:
iid_dict = dict(zip(item.item_id, item.index))

In [35]:
item['new_id'] = item.index
item.drop(['item_id'], axis=1, inplace=True)
item.rename(columns={'new_id': 'item_id'}, inplace=True)
# item = item[['item_id', 'item_picture', 'item_cluster1', 'item_cluster2']]

In [25]:
# item.nunique()
item

,item_picture,item_cluster1,item_cluster2,keywords,keywords_p,item_id
0,145,26,226,"[898908, 908461, 769346, 668886, 607775, 48537...","[8.469502, 8.137443, 9.022247, 11.218712, 5.10...",0
1,2,5,57,"[630935, 304133, 230788, 317745, 496902, 75627...","[14.760494, 16.474872, 15.597788, 11.979615, 1...",1
2,145,26,221,"[462756, 886628, 667018, 779609, 634056, 17889...","[8.979802, 7.951116, 5.954278, 7.046073, 6.572...",2
3,68,26,221,"[752149, 1018136, 551800, 498393, 199619, 3135...","[12.055207, 8.878175, 5.342025, 9.453552, 12.5...",3
4,72,26,221,"[887532, 887529, 57198, 46815, 105704, 929252,...","[12.772149, 8.394001, 8.436843, 7.750292, 9.84...",4
...,...,...,...,...,...,...
633386,134,31,264,"[680565, 969246, 772647, 392180, 473471, 60425...","[6.528161, 6.471880, 7.514914, 10.299104, 9.16...",633386
633387,68,4,33,"[133868, 766834, 863966, 531254, 66951, 711935...","[5.064578, 14.577604, 24.509433, 8.474139, 10....",633387
633388,123,19,180,"[635058, 806578, 358461, 344530, 413663, 26961...","[7.489791, 18.557439, 8.061633, 8.413115, 11.6...",633388
633389,3,4,7,"[760967, 965192, 540951, 989730, 1019867, 5530...","[16.306715, 9.342195, 8.706187, 10.458268, 8.1...",633389


In [ ]:
# 存item_feature
with open(r'./item_feature1.pkl', 'wb') as f:
    pickle.dump(item, f, pickle.HIGHEST_PROTOCOL)

## 处理训练集和测试集

In [17]:
train.head(10)

,user_id,item_id,time,network,refresh,position,label,duration
0,1000014754,463510256,1624843756147,5,0,16,0,0
1,1000014754,463852707,1624843756147,5,0,13,1,80
2,1000014754,464757134,1625052999841,5,0,13,1,1050
3,1000014754,464617167,1625052999841,5,0,16,1,286
4,1000014754,465426190,1625382421168,5,0,5,0,0
5,1000014754,465815972,1625382421168,5,0,4,1,285
6,1000014754,465991958,1625382421168,5,0,0,1,353
7,1000014754,463067100,1624757147178,5,0,13,0,0
8,1000014754,464264603,1625011694982,5,1,18,0,0
9,1000014754,464295109,1625009731701,5,0,19,1,215


In [20]:
train[['user_id', 'item_id', 'network', 'refresh', 'label']].nunique()

user_id    1478694
item_id     590949
network          4
refresh        651
label            2
dtype: int64

- 训练集的结束时间为2021.6.24 22:56:53, 训练集的结束时间为2021.7.6 23:59:59
- 测试集的结束时间为2021.7.7 00:00:04, 训练集的结束时间为2021.7.7 23:41:22
- 验证集截取7.6一天的数据

In [26]:
with open(r'./train_online.pkl', 'wb') as f:
    pickle.dump(train[['user_id', 'item_id', 'network', 'refresh', 'label']].to_numpy(), f, pickle.HIGHEST_PROTOCOL)

In [36]:
validation = train.loc[train['time']>1625500811000]
validation.shape

(15192000, 8)

In [37]:
train = train.loc[set(train.index) - set(validation.index)]
train.shape

(174574959, 8)

In [38]:
train2 = train[['user_id', 'item_id', 'network', 'refresh', 'label']]
validation = validation[['user_id', 'item_id', 'network', 'refresh', 'label']]
test = test[['user_id', 'item_id', 'network', 'refresh']]

# 存储数据

In [30]:
with open(r'./train.pkl', 'wb') as f:
    pickle.dump(train2.to_numpy(), f, pickle.HIGHEST_PROTOCOL)
with open(r'./test.pkl', 'wb') as f:
    pickle.dump(test.to_numpy(), f, pickle.HIGHEST_PROTOCOL)
with open(r'./validation.pkl', 'wb') as f:
    pickle.dump(validation.to_numpy(), f, pickle.HIGHEST_PROTOCOL)
with open(r'./id_dict.pkl', 'wb') as f:
    pickle.dump((uid_dict, iid_dict), f, pickle.HIGHEST_PROTOCOL)
    

In [56]:
new_user[['inter_count', 'refresh_mean', 'refresh_max', 'refresh_min','consumption_mean', 'consumption_max', 'consumption_min']].nunique()

inter_count          9
refresh_mean        10
refresh_max         11
refresh_min          8
consumption_mean     3
consumption_max      3
consumption_min      8
dtype: int64

In [57]:
new_user['user_device'].nunique()

3096

In [39]:
import json
info_dict = {'feat_field': {}, 'vocabulary_size':{}, 'feat_type':{'dense':[], 'sparse':[], 'multi-hot':[]}, 'sample_num': train2.shape[0], 'user_num': new_user.shape[0], 'item_num': item.shape[0]}

user_feats = list(new_user.columns)
user_feats.remove('user_age')
user_feats.remove('user_gender')
for feat in user_feats:
    if feat in ['inter_count', 'refresh_mean', 'refresh_max', 'refresh_min','consumption_mean', 'consumption_max', 'consumption_min']:
        info_dict['vocabulary_size'][feat] = 11
    else:
        info_dict['vocabulary_size'][feat] = new_user[feat].nunique()
info_dict['vocabulary_size']['user_age'] = 4
info_dict['vocabulary_size']['user_gender'] = 2

item_feats = list(item.columns)
item_feats.remove('keywords')
item_feats.remove('keywords_p')
for feat in item_feats:
    info_dict['vocabulary_size'][feat] = item[feat].nunique()
info_dict['vocabulary_size']['keywords'] = len(keywords)+1
# 需要+1 因为embedding dict编号从零开始

for feat in ['network', 'refresh']:
    info_dict['vocabulary_size'][feat] = train2[feat].nunique()
info_dict['vocabulary_size']['network'] = 6
info_dict['feat_type']['sparse'] = user_feats + item_feats + ['network', 'refresh']
info_dict['feat_type']['multi-hot'] = ['user_age', 'user_gender', 'keywords']
info_dict['feat_field']['user'] = list(new_user.columns)
info_dict['feat_field']['item'] = list(item.columns)
info_dict['feat_field']['inter'] = list(train2.columns)

with open(r'./info2-2.json', 'w') as f:
    json.dump(info_dict, f)

In [ ]:
info2-2 线上线下均可使用